# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
# Check for missing values
print(spaceship.isnull().sum())

# Drop rows with missing values for simplicity (adjust based on your previous approach)
spaceship.dropna(inplace=True)

# Transform the Cabin feature to its first letter
spaceship['Cabin'] = spaceship['Cabin'].str[0].str.upper()

# Drop PassengerId and Name columns
spaceship.drop(columns=['PassengerId', 'Name'], inplace=True)

# Create dummy variables for categorical columns
spaceship = pd.get_dummies(spaceship, drop_first=True)

# Split features and target variable
X = spaceship.drop(columns=['Transported'])  # Adjust as needed
y = spaceship['Transported']

# Scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


**Perform Train Test Split**

In [4]:
# Perform Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Create a Bagging model
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
bagging_model.fit(X_train, y_train)

# Evaluate the model
bagging_accuracy = bagging_model.score(X_test, y_test)
print(f"Bagging Model Accuracy: {bagging_accuracy:.2f}")


Bagging Model Accuracy: 0.80


- Random Forests

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
rf_accuracy = rf_model.score(X_test, y_test)
print(f"Random Forest Model Accuracy: {rf_accuracy:.2f}")


Random Forest Model Accuracy: 0.81


- Gradient Boosting

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Evaluate the model
gb_accuracy = gb_model.score(X_test, y_test)
print(f"Gradient Boosting Model Accuracy: {gb_accuracy:.2f}")


Gradient Boosting Model Accuracy: 0.81


- Adaptive Boosting

In [9]:
from sklearn.ensemble import AdaBoostClassifier

# Create an AdaBoost model
ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

# Evaluate the model
ab_accuracy = ab_model.score(X_test, y_test)
print(f"AdaBoost Model Accuracy: {ab_accuracy:.2f}")


c:\Users\Marc Jay\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Model Accuracy: 0.80


Which model is the best and why?

In [11]:
print(f"Bagging Accuracy: {bagging_accuracy:.2f}")
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")
print(f"Gradient Boosting Accuracy: {gb_accuracy:.2f}")
print(f"AdaBoost Accuracy: {ab_accuracy:.2f}")




# Which model is the best and why?
best_model = max(bagging_accuracy, rf_accuracy, gb_accuracy, ab_accuracy)
if best_model == rf_accuracy:
    print("The Random Forest model performed the best due to its ability to reduce overfitting through averaging multiple decision trees.")
elif best_model == gb_accuracy:
    print("The Gradient Boosting model performed the best, leveraging weak learners to boost performance on difficult cases.")
elif best_model == ab_accuracy:
    print("The AdaBoost model performed the best, focusing on misclassified instances to improve accuracy.")
else:
    print("The Bagging model performed the best, enhancing model stability and accuracy.")


Bagging Accuracy: 0.80
Random Forest Accuracy: 0.81
Gradient Boosting Accuracy: 0.81
AdaBoost Accuracy: 0.80
The Gradient Boosting model performed the best, leveraging weak learners to boost performance on difficult cases.
